In [30]:
"""
- 학습할때는 encoder, decoder를 함께 학습을 하고, 
- 나중에 사용할 때는 encoder, decoder를 다른 모델로 세팅해서 만드는 것으로 보임

- model과 Sequential은 어떻게 다른가?? 왜 에러가 발생하는가?? 
    - model에서는 보통 input과 output만을 넣어줌. 
"""
from keras.layers import Input, Dense
from keras.models import Model

# 데이터를 몇 차원으로 줄일 것인지
encoding_dim = 32

input_img = Input(shape=(784,))
encoded = Dense(encoding_dim, activation='relu')(input_img)
print(type(encoded))
# "decoded" is the lossy reconstruction of the input
decoded = Dense(784, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)
autoencoder.summary()

# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)
encoder.summary()

encoded_input = Input(shape=(encoding_dim,))

# create decoder model 
decoder_layer = autoencoder.layers[-1]
"""
- 그냥 decoder_layer = decoded 로 해도 될것 이라고 생각했는데 잘 안됨. 
- print(autoencoder.layers[-1]==decoded) 라고 해도, False로 나옴 
- 다양한 이유가 있을 수 있겠지만, decoded의 경우는 이미 input layer까지 정해져 있는 상황임. 
- decoder에서는 parameter만 가져오고, 새로운 input layer를 결정해주어야 하는 것이라서, 아마도 다르게 나오는 것으로 보임. 
"""
decoder = Model(encoded_input, decoder_layer(encoded_input))
decoder.summary()

<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_39 (InputLayer)        (None, 784)               0         
_________________________________________________________________
dense_53 (Dense)             (None, 32)                25120     
_________________________________________________________________
dense_54 (Dense)             (None, 784)               25872     
Total params: 50,992
Trainable params: 50,992
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_39 (InputLayer)        (None, 784)               0         
_________________________________________________________________
dense_53 (Dense)             (None, 32) 

In [28]:
"""
- 위의 코드를 아래의 방식으로 새롭게 정의해줄 수 있지 않나 싶은데. 
"""
from keras.layers import Input, Dense
from keras.models import Model, Sequential

# 데이터를 몇 차원으로 줄일 것인지
encoding_dim = 32

input_img = Input(shape=(784,))

encoded = Dense(encoding_dim, activation='relu')
decoded = Dense(784, activation='sigmoid')
autoencoder = Sequential()
autoencoder.add(input_img)
autoencoder.summary()

# this model maps an input to its encoded representation
encoder = Sequential([input_img, encoded])
encoder.summary()

decoder = Sequential([Input(shape=(encoding_dim,)), decoded])
# create decoder model 
decoder_layer = autoencoder.layers[-1]
decoder.summary()

TypeError: The added layer must be an instance of class Layer. Found: Tensor("input_36:0", shape=(?, 784), dtype=float32)

In [47]:
"""
- class를 생성했으나, 내부에 __init__는 없고 __call__만 있음
- 객체로 표현했으나, 실제는 함수처럼 사용되는 경우, 이러한 경우를 callable object라고 하는 듯. ㅌㅁ
"""
class Foo(object):
    def __call__(self):
        print("Called.")
foo_instance = Foo()
print(type(foo_instance))
foo_instance()

<class '__main__.Foo'>
Called.


In [51]:
class Accumulator(object):
    def __init__(self, n):
        self.n = n
    def __call__(self, x):
        self.n += x
        return self.n

from_1 = Accumulator(1)
from_10 = Accumulator(10)

print(from_1(10))
print(from_10(200))

11
210


In [45]:
type(a)

__main__.Accumulator